In [ ]:
%pip install -U datasets
%pip install transformers
%pip install torch
%pip install sacrebleu
%pip install tqdm
%pip install numpy
%pip install regex
%pip install accelerate
%pip install bert-score
%pip install sentence-transformers
%pip install unbabel-comet

In [ ]:
from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from tqdm import tqdm
import logging
from sacrebleu import corpus_bleu
import os
import json
from sacrebleu.metrics import BLEU, CHRF
from bert_score import score as bert_score
from sentence_transformers import SentenceTransformer
import numpy as np
from collections import Counter
import re
import requests
from datasets import Dataset
from comet import download_model, load_from_checkpoint
import zipfile
import os
from datetime import datetime

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA (NVIDIA GPU) for acceleration")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Metal Performance Shaders) for acceleration")
else:
    device = torch.device("cpu")
    print("No GPU acceleration available, using CPU")

In [ ]:
def fetch_data_from_github(url):
    response = requests.get(url)
    if response.status_code == 200:
        return [line.strip() for line in response.text.splitlines()]
    else:
        print(f"Failed to fetch data from {url}")
        return []

In [ ]:
# folders to store translations

os.makedirs("translations", exist_ok=True)

os.makedirs("translations/nllb-200-distilled-600M", exist_ok=True)

os.makedirs("translations/m2m100_418M", exist_ok=True)

# folders to store refrences for flores 101

os.makedirs("flores101", exist_ok=True)

os.makedirs("flores101/english-sources", exist_ok=True)

os.makedirs("flores101/northern-sotho", exist_ok=True)

os.makedirs("flores101/hausa", exist_ok=True)

os.makedirs("flores101/zulu", exist_ok=True)

# folders to store refrences flores 200

os.makedirs("flores200", exist_ok=True)

os.makedirs("flores200/english-sources", exist_ok=True)

os.makedirs("flores200/northern-sotho", exist_ok=True)

os.makedirs("flores200/hausa", exist_ok=True)

os.makedirs("flores200/zulu", exist_ok=True)

# folders to store refrences for the fix for africa

os.makedirs("floresfixforafrica", exist_ok=True)

os.makedirs("floresfixforafrica/english-sources", exist_ok=True)

os.makedirs("floresfixforafrica/northern-sotho", exist_ok=True)

os.makedirs("floresfixforafrica/hausa", exist_ok=True)

os.makedirs("floresfixforafrica/zulu", exist_ok=True)

#creating metrics folders
os.makedirs("metrics", exist_ok=True)

#metrics folders for nllb
os.makedirs("metrics/nllb-200-distilled-600M", exist_ok=True)

os.makedirs("metrics/nllb-200-distilled-600M/flores101", exist_ok=True)

os.makedirs("metrics/nllb-200-distilled-600M/flores200", exist_ok=True)

os.makedirs("metrics/nllb-200-distilled-600M/floresfixforafrica", exist_ok=True)

#metrics folders for m2m
os.makedirs("metrics/m2m100_418M", exist_ok=True)

os.makedirs("metrics/m2m100_418M/flores101", exist_ok=True)

os.makedirs("metrics/m2m100_418M/flores200", exist_ok=True)

os.makedirs("metrics/m2m100_418M/floresfixforafrica", exist_ok=True)

In [ ]:
#english
eng_dataset_test = load_dataset("gsarti/flores_101", name="eng", split="devtest", cache_dir=None)
eng_dataset_training = load_dataset("gsarti/flores_101", name="eng", split="dev", cache_dir=None)

#nothern sotho
nso_dataset_flores_101_test = load_dataset("gsarti/flores_101", name="nso", split="devtest", cache_dir=None)
nso_dataset_flores_101_training = load_dataset("gsarti/flores_101", name="nso", split="dev", cache_dir=None)

#hausa
hau_dataset_flores_101_test = load_dataset("gsarti/flores_101", name="hau", split="devtest", cache_dir=None)
hau_dataset_flores_101_training = load_dataset("gsarti/flores_101", name="hau", split="dev", cache_dir=None)

#zulu
zul_dataset_flores_101_test = load_dataset("gsarti/flores_101", name="zul", split="devtest", cache_dir=None)
zul_dataset_flores_101_training = load_dataset("gsarti/flores_101", name="zul", split="dev", cache_dir=None)

In [ ]:
#english
eng_dataset_flores_200_test = load_dataset("Muennighoff/flores200", name="eng_Latn", split="devtest", trust_remote_code=True, cache_dir=None)
eng_dataset_flores_200_training = load_dataset("Muennighoff/flores200", name="eng_Latn", split="dev", cache_dir=None)

#nothern sotho
nso_dataset_flores_200_test = load_dataset("Muennighoff/flores200", name="nso_Latn", split="devtest", trust_remote_code=True, cache_dir=None)
nso_dataset_flores_200_training = load_dataset("Muennighoff/flores200", name="nso_Latn", split="dev", cache_dir=None)

#hausa
hau_dataset_flores_200_test = load_dataset("Muennighoff/flores200", name="hau_Latn", split="devtest", trust_remote_code=True, cache_dir=None)
hau_dataset_flores_200_training = load_dataset("Muennighoff/flores200", name="hau_Latn", split="dev", cache_dir=None)

#zulu
zul_dataset_flores_200_test = load_dataset("Muennighoff/flores200", name="zul_Latn", split="devtest", trust_remote_code=True, cache_dir=None)
zul_dataset_flores_200_training = load_dataset("Muennighoff/flores200", name="zul_Latn", split="dev", trust_remote_code=True, cache_dir=None)

In [ ]:
# data links
zul_dev_url = "https://raw.githubusercontent.com/dsfsi/flores-fix-4-africa/main/data/corrected/dev/zul_Latn.dev"
zul_devtest_url = "https://raw.githubusercontent.com/dsfsi/flores-fix-4-africa/main/data/corrected/devtest/zul_Latn.devtest"

nso_dev_url = "https://raw.githubusercontent.com/dsfsi/flores-fix-4-africa/main/data/corrected/dev/nso_Latn.dev"
nso_devtest_url = "https://raw.githubusercontent.com/dsfsi/flores-fix-4-africa/main/data/corrected/devtest/nso_Latn.devtest"

hau_dev_url = "https://raw.githubusercontent.com/dsfsi/flores-fix-4-africa/main/data/corrected/dev/hau_Latn.dev"
hau_devtest_url = "https://raw.githubusercontent.com/dsfsi/flores-fix-4-africa/main/data/corrected/devtest/hau_Latn.devtest"

#zulu
zul_dataset_flores_fix_for_africa_test = fetch_data_from_github(zul_devtest_url)
zul_dataset_flores_fix_for_africa_training = fetch_data_from_github(zul_dev_url)

#nothern sotho
nso_dataset_flores_fix_for_africa_test = fetch_data_from_github(nso_devtest_url)
nso_dataset_flores_fix_for_africa_training = fetch_data_from_github(nso_dev_url)

#hausa
hau_dataset_flores_fix_for_africa_test = fetch_data_from_github(hau_devtest_url)
hau_dataset_flores_fix_for_africa_training = fetch_data_from_github(hau_dev_url)

In [ ]:
assert len(eng_dataset_test) == len(nso_dataset_flores_101_test), "Mismatched test dataset sizes"
assert len(eng_dataset_training) == len(nso_dataset_flores_101_training), "Mismatched training dataset sizes"
assert len(eng_dataset_test) == len(zul_dataset_flores_101_test), "Mismatched test dataset sizes"
assert len(eng_dataset_training) == len(zul_dataset_flores_101_training), "Mismatched training dataset sizes"
assert len(eng_dataset_test) == len(hau_dataset_flores_101_test), "Mismatched test dataset sizes"
assert len(eng_dataset_training) == len(hau_dataset_flores_101_training), "Mismatched training dataset sizes"

In [ ]:
assert len(eng_dataset_test) == len(nso_dataset_flores_fix_for_africa_test), "Mismatched test dataset sizes"
assert len(eng_dataset_training) == len(nso_dataset_flores_fix_for_africa_training), "Mismatched training dataset sizes"
assert len(eng_dataset_test) == len(zul_dataset_flores_fix_for_africa_test), "Mismatched test dataset sizes"
assert len(eng_dataset_training) == len(zul_dataset_flores_fix_for_africa_training), "Mismatched training dataset sizes"
assert len(eng_dataset_test) == len(hau_dataset_flores_fix_for_africa_test), "Mismatched test dataset sizes"
assert len(eng_dataset_training) == len(hau_dataset_flores_fix_for_africa_training), "Mismatched training dataset sizes"

In [ ]:
# English Sources for flores 101
with open("flores101/english-sources/flores101.english.source.test.txt", "w", encoding="utf-8") as f:
    f.writelines(line["sentence"] + "\n" for line in eng_dataset_test)

with open("flores101/english-sources/flores101.english.source.training.txt", "w", encoding="utf-8") as f:
    f.writelines(line["sentence"] + "\n" for line in eng_dataset_training)

# English Sources for fix for africa
with open("floresfixforafrica/english-sources/floresfixforafrica.english.source.test.txt", "w", encoding="utf-8") as f:
    f.writelines(line["sentence"] + "\n" for line in eng_dataset_test)

with open("floresfixforafrica/english-sources/floresfixforafrica.english.source.training.txt", "w", encoding="utf-8") as f:
    f.writelines(line["sentence"] + "\n" for line in eng_dataset_training)



In [ ]:
# English Sources for flores 200
with open("flores200/english-sources/flores200.english.source.test.txt", "w", encoding="utf-8") as f:
    f.writelines(line["sentence"] + "\n" for line in eng_dataset_flores_200_test)

with open("flores200/english-sources/flores200.english.source.training.txt", "w", encoding="utf-8") as f:
    f.writelines(line["sentence"] + "\n" for line in eng_dataset_flores_200_training)

In [ ]:
# Northern Sotho

with open("flores101/northern-sotho/flores101.northern-sotho.ref.test.txt", "w", encoding="utf-8") as f:
    f.writelines(line["sentence"] + "\n" for line in nso_dataset_flores_101_test)

with open("flores101/northern-sotho/flores101.northern-sotho.ref.training.txt", "w", encoding="utf-8") as f:
    f.writelines(line["sentence"] + "\n" for line in nso_dataset_flores_101_training)

# Hausa

with open("flores101/hausa/flores101.hausa.ref.test.txt", "w", encoding="utf-8") as f:
    f.writelines(line["sentence"] + "\n" for line in hau_dataset_flores_101_test)

with open("flores101/hausa/flores101.hausa.ref.training.txt", "w", encoding="utf-8") as f:
    f.writelines(line["sentence"] + "\n" for line in hau_dataset_flores_101_training)

# Zulu
with open("flores101/zulu/flores101.zulu.ref.test.txt", "w", encoding="utf-8") as f:
    f.writelines(line["sentence"] + "\n" for line in zul_dataset_flores_101_test)

with open("flores101/zulu/flores101.zulu.ref.training.txt", "w", encoding="utf-8") as f:
    f.writelines(line["sentence"] + "\n" for line in zul_dataset_flores_101_training)


In [ ]:
# Northern Sotho

with open("floresfixforafrica/northern-sotho/floresfixforafrica.northern-sotho.ref.test.txt", "w", encoding="utf-8") as f:
    f.writelines(line + "\n" for line in nso_dataset_flores_fix_for_africa_test)

with open("floresfixforafrica/northern-sotho/floresfixforafrica.northern-sotho.ref.training.txt", "w", encoding="utf-8") as f:
    f.writelines(line + "\n" for line in nso_dataset_flores_fix_for_africa_training)

# Hausa

with open("floresfixforafrica/hausa/floresfixforafrica.hausa.ref.test.txt", "w", encoding="utf-8") as f:
    f.writelines(line + "\n" for line in hau_dataset_flores_fix_for_africa_test)

with open("floresfixforafrica/hausa/floresfixforafrica.hausa.ref.training.txt", "w", encoding="utf-8") as f:
    f.writelines(line + "\n" for line in hau_dataset_flores_fix_for_africa_training)

# Zulu

with open("floresfixforafrica/zulu/floresfixforafrica.zulu.ref.test.txt", "w", encoding="utf-8") as f:
    f.writelines(line + "\n" for line in zul_dataset_flores_fix_for_africa_test)

with open("floresfixforafrica/zulu/floresfixforafrica.zulu.ref.training.txt", "w", encoding="utf-8") as f:
    f.writelines(line + "\n" for line in zul_dataset_flores_fix_for_africa_training)

In [ ]:
# Northern Sotho

with open("flores200/northern-sotho/flores200.northern-sotho.ref.test.txt", "w", encoding="utf-8") as f:
    f.writelines(line["sentence"] + "\n" for line in nso_dataset_flores_200_test)

with open("flores200/northern-sotho/flores200.northern-sotho.ref.training.txt", "w", encoding="utf-8") as f:
    f.writelines(line["sentence"] + "\n" for line in nso_dataset_flores_200_training)

# Hausa

with open("flores200/hausa/flores200.hausa.ref.test.txt", "w", encoding="utf-8") as f:
    f.writelines(line["sentence"] + "\n" for line in hau_dataset_flores_200_test)

with open("flores200/hausa/flores200.hausa.ref.training.txt", "w", encoding="utf-8") as f:
    f.writelines(line["sentence"] + "\n" for line in hau_dataset_flores_200_training)

# Zulu

with open("flores200/zulu/flores200.zulu.ref.test.txt", "w", encoding="utf-8") as f:
    f.writelines(line["sentence"] + "\n" for line in zul_dataset_flores_200_test)

with open("flores200/zulu/flores200.zulu.ref.training.txt", "w", encoding="utf-8") as f:
    f.writelines(line["sentence"] + "\n" for line in zul_dataset_flores_200_training)

In [ ]:
def load_and_prepare_data(target_lang, split="dev"):
    """Load and prepare the FLORES-101 dataset for the specified language."""
    try:
        dataset_name = "facebook/flores"
        source_lang = "eng_Latn"

        # Map language codes to their full format with script
        lang_code_map = {
            "hau": "hau_Latn",
            "nso": "nso_Latn",
            "zul": "zul_Latn"
        }

        target_lang_full = lang_code_map.get(target_lang)
        if not target_lang_full:
            raise ValueError(f"Unsupported target language code: {target_lang}")

        # Load source and target datasets
        source_dataset = load_dataset(dataset_name, name=source_lang, split=split, trust_remote_code=True)
        target_dataset = load_dataset(dataset_name, name=target_lang_full, split=split, trust_remote_code=True)

        # Create training pairs
        training_data = {
            "input_text": [src["sentence"] for src in source_dataset],
            "target_text": [tgt["sentence"] for tgt in target_dataset]
        }

        # Convert to HuggingFace Dataset
        from datasets import Dataset
        return Dataset.from_dict(training_data)
    except Exception as e:
        print(f"Error loading dataset: {e}")
        raise


In [ ]:
def initialize_model(model_path="facebook/nllb-200-distilled-600M"):
    """Initialize the NLLB model and tokenizer."""
    try:
        # Check for MPS (Metal Performance Shaders) availability
        if torch.cuda.is_available():
            device = torch.device("cuda")
            print("Using CUDA (NVIDIA GPU) for acceleration")
        elif torch.backends.mps.is_available():
            device = torch.device("mps")
            print("Using MPS (Metal Performance Shaders) for acceleration")
        else:
            device = torch.device("cpu")
            print("No GPU acceleration available, using CPU")

        # Load model and tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)

        return tokenizer, model, device
    except Exception as e:
        print(f"Error initializing model: {e}")
        raise


In [ ]:
def translate_english_to_target_lang(model, tokenizer, device, ref_sentences, output_file, target_lang_code, lang_codes, batch_size=16):
    """Translate English sentences to target language using the pre trained model."""
    try:

        target_lang = lang_codes.get(target_lang_code)
        if not target_lang:
            raise ValueError(f"Unsupported target language code: {target_lang_code}")

        forced_bos_token_id = tokenizer.convert_tokens_to_ids(target_lang)

        print(f"Starting translation to {target_lang}...")

        # Prepare all sentences for batch processing
        all_sentences = ref_sentences["input_text"]

        # Ensure output directory exists
        os.makedirs(os.path.dirname(output_file), exist_ok=True)

        with open(output_file, "w", encoding="utf-8") as f:
            # Process in batches
            for i in tqdm(range(0, len(all_sentences), batch_size), desc=f"Translating to {target_lang}"):
                batch_sentences = all_sentences[i:i + batch_size]

                # Tokenize batch without length limits
                inputs = tokenizer(
                    batch_sentences,
                    return_tensors="pt",
                    padding=True,
                    truncation=False
                ).to(device)

                with torch.no_grad():
                    translated = model.generate(
                        **inputs,
                        forced_bos_token_id=forced_bos_token_id,
                        num_beams=4,
                        early_stopping=True
                    )

                # Decode and write batch results
                decoded = tokenizer.batch_decode(translated, skip_special_tokens=True)
                for translation in decoded:
                    f.write(translation.strip() + "\n")

        print(f"Translation completed. Results saved to {output_file}")
    except Exception as e:
        print(f"Error during translation: {e}")
        raise


In [ ]:
def calculate_metrics(hypotheses, references, sources, lang, device):
    """Calculate various translation metrics including COMET.

    Args:
        hypotheses: List of translated texts (machine translations)
        references: List of reference translations (human translations)
        sources: List of source texts (original texts in source language)
        lang: Language code (e.g., 'en', 'de')
        device: Computation device ('cpu' or 'cuda')
    """
    metrics = {}

    try:
        # BLEU score
        bleu = BLEU()
        metrics['BLEU'] = bleu.corpus_score(hypotheses, [references]).score

        # chrF score
        chrf = CHRF()
        metrics['chrF'] = chrf.corpus_score(hypotheses, [references]).score

        # BERTScore
        P, R, F1 = bert_score(hypotheses, references, lang=lang, device=device)
        metrics['BERTScore'] = F1.mean().item()

        # COMET score (masakhane/africomet-mtl)
        comet_model_path = download_model("masakhane/africomet-mtl")
        comet_model = load_from_checkpoint(comet_model_path)

        comet_data = [{'src': src, 'mt': hyp, 'ref': ref}
                     for src, hyp, ref in zip(sources, hypotheses, references)]

        comet_scores = comet_model.predict(
            comet_data,
            batch_size=8,
            gpus=1 if torch.cuda.is_available() else 0
        )

        metrics['COMET'] = comet_scores.system_score

        # # Semantic Similarity Score (optional)
        # model = SentenceTransformer('all-MiniLM-L6-v2')
        # hyp_embeddings = model.encode(hypotheses, convert_to_tensor=True)
        # ref_embeddings = model.encode(references, convert_to_tensor=True)
        # similarity = torch.nn.functional.cosine_similarity(hyp_embeddings, ref_embeddings)
        # metrics['Semantic_Score'] = similarity.mean().item()

        # Error Type Frequency Distribution
        error_types = analyze_errors(hypotheses, references)
        metrics['Error_Distribution'] = error_types

    except Exception as e:
        print(f"Error calculating metrics: {e}")
        metrics['error'] = str(e)

    return metrics


In [ ]:
def analyze_errors(hypotheses, references):
    """Analyze translation errors and their distribution."""
    error_types = Counter()

    for hyp, ref in zip(hypotheses, references):
        # Word order errors
        hyp_words = set(hyp.split())
        ref_words = set(ref.split())
        if hyp_words == ref_words and hyp != ref:
            error_types['word_order'] += 1

        # Missing words
        missing = ref_words - hyp_words
        if missing:
            error_types['missing_words'] += len(missing)

        # Extra words
        extra = hyp_words - ref_words
        if extra:
            error_types['extra_words'] += len(extra)

        # Case errors
        if hyp.lower() == ref.lower() and hyp != ref:
            error_types['case_errors'] += 1

        # Punctuation errors
        hyp_no_punct = re.sub(r'[^\w\s]', '', hyp)
        ref_no_punct = re.sub(r'[^\w\s]', '', ref)
        if hyp_no_punct == ref_no_punct and hyp != ref:
            error_types['punctuation_errors'] += 1

    return dict(error_types)


In [ ]:
# Function to save metrics to a file
def save_metrics_to_file(metrics, lang, datasetname, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    metrics_file = f"{output_dir}/{datasetname}/{lang}_metrics.json"
    print(metrics_file)

    with open(metrics_file, "w", encoding="utf-8") as f:
        json.dump(metrics, f, ensure_ascii=False, indent=4)

In [ ]:
# Function to load saved metrics from a file
def load_metrics_from_file(lang, datasetname, output_dir):
    metrics_file = f"{output_dir}/{datasetname}/{lang}_metrics.json"
    if os.path.exists(metrics_file):
        with open(metrics_file, "r", encoding="utf-8") as f:
            return json.load(f)
    return None  # Return None if no metrics file exists

In [ ]:
def create_dataset_from_sentences(source_dataset, target_dataset):
    """
    Creates a Hugging Face Dataset from source and target sentence pairs.

    Parameters:
    - source_dataset (list of dicts): The source language dataset, e.g., English.
    - target_dataset (list of sentences): The target language dataset, e.g., Hausa.

    Returns:
    - A Hugging Face Dataset object containing 'input_text' and 'target_text'.
    """
    # Ensure the datasets are non-empty and have matching lengths
    if len(source_dataset) != len(target_dataset):
        raise ValueError("Source and target datasets must have the same length.")

    # Create pairs of 'input_text' and 'target_text'
    training_data = {
        "input_text": [src["sentence"] for src in source_dataset],
        "target_text": [tgt for tgt in target_dataset]
    }

    # Convert to Hugging Face Dataset
    return Dataset.from_dict(training_data)


In [ ]:
def create_dataset_from_sentences_with_dict(source_dataset, target_dataset):
    """
    Creates a Hugging Face Dataset from source and target sentence pairs.

    Parameters:
    - source_dataset (list of dicts): The source language dataset, e.g., English.
    - target_dataset (list of sentences): The target language dataset, e.g., Hausa.

    Returns:
    - A Hugging Face Dataset object containing 'input_text' and 'target_text'.
    """
    # Ensure the datasets are non-empty and have matching lengths
    if len(source_dataset) != len(target_dataset):
        raise ValueError("Source and target datasets must have the same length.")

    # Create pairs of 'input_text' and 'target_text'
    training_data = {
        "input_text": [src["sentence"] for src in source_dataset],
        "target_text": [tgt["sentence"] for tgt in target_dataset]
    }

    # Convert to Hugging Face Dataset
    return Dataset.from_dict(training_data)

In [ ]:
#translation by nllb for each language

# Configuration
languages = {
    "hausa": "hau",
    "northern-sotho": "nso",
    "zulu": "zul"
}

models = [
    {
        "nllb-200-distilled-600M" : "facebook/nllb-200-distilled-600M",
        "lang_codes" : {
            "hau": "hau_Latn",
            "nso": "nso_Latn",
            "zul": "zul_Latn"
        }
    },
    {
        "m2m100_418M" : "facebook/m2m100_418M",
        "lang_codes" : {
            "hau": "ha",
            "nso": "ns",
            "zul": "zu"
        }
    }
]

translations_base_dir = "translations"

for model in models:
  model_name = list(model.keys())[0]  # Get the model name
  model_path = model[model_name]      # Get the Hugging Face path
  lang_codes = model["lang_codes"]    # Get the language codes dict

  print(f"\n Translating using the Model: {model_name}")

  # Initialize model and tokenizer
  tokenizer, model, device = initialize_model(model_path)

  # Translate for each language
  for lang, code in languages.items():
      translations_file = f"{translations_base_dir}/{model_name}/{lang}.hyp.txt"

      # Check if translation already exists
      if os.path.exists(translations_file) and os.path.getsize(translations_file) > 0:
          print(f"Translation for {lang} already exists. Skipping translation...")
      else:
          print(f"Translating to {lang}...")
          if(lang == "hausa"):
            test_data = create_dataset_from_sentences_with_dict(eng_dataset_test, hau_dataset_flores_101_test)
            translate_english_to_target_lang(model, tokenizer, device, test_data, translations_file, code, lang_codes)
          elif(lang == "zulu"):
            test_data = create_dataset_from_sentences_with_dict(eng_dataset_test, zul_dataset_flores_101_test)
            translate_english_to_target_lang(model, tokenizer, device, test_data, translations_file, code, lang_codes)
          elif(lang == "northern-sotho"):
            test_data = create_dataset_from_sentences_with_dict(eng_dataset_test, nso_dataset_flores_101_test)
            translate_english_to_target_lang(model, tokenizer, device, test_data, translations_file, code, lang_codes)

In [ ]:
flores_datasets = [
    "flores101",
    "flores200",
    "floresfixforafrica",
]

for model in models:
  model_name = list(model.keys())[0]  # Get the model name
  model_path = model[model_name]      # Get the Hugging Face path
  lang_codes = model["lang_codes"]    # Get the language codes dict

  for flores_dataset in flores_datasets:
    #calculate metric for each language
    for lang, code in languages.items():
        translations_file = f"{translations_base_dir}/{model_name}/{lang}.hyp.txt"

        # Check if metrics are already saved
        metrics = load_metrics_from_file(lang, flores_dataset,f"metrics/{model_name}")

        if not metrics:
            # Calculate metrics
            try:
                with open(translations_file, "r", encoding="utf-8") as f:
                    hyps = [line.strip() for line in f]

                with open(f"{flores_dataset}/{lang}/{flores_dataset}.{lang}.ref.test.txt", "r", encoding="utf-8") as f:
                    refs = [line.strip() for line in f]

                with open(f"{flores_dataset}/english-sources/{flores_dataset}.english.source.test.txt", "r", encoding="utf-8") as f:
                    sources = [line.strip() for line in f]

                metrics = calculate_metrics(hyps, refs, sources, lang, device)

                # Save metrics to file
                save_metrics_to_file(metrics,lang, flores_dataset, f"metrics/{model_name}")

                print(f"\nMetrics for {lang} using {flores_dataset} dataset and {model_name} for translation:")
                print("-" * 30)
                for metric_name, value in metrics.items():
                    if metric_name != 'Error_Distribution':
                        print(f"{metric_name:15}: {value:.4f}")

                if 'Error_Distribution' in metrics:
                    print("\nError Distribution:")
                    for error_type, count in metrics['Error_Distribution'].items():
                        print(f"{error_type:20}: {count}")

            except Exception as e:
                print(f"Error calculating metrics for {lang}: {e}")
                print(f"Could not calculate metrics for {lang}")

    # Print summary of all metrics
    print(f"\nSummary of all metrics for {flores_dataset} using {model_name} for translation:")
    print("-" * 60)
    print(f"{'Language':15} {'BLEU':>8} {'chrF':>8} {'BERTScore':>10} {'COMET':>8}")
    print("-" * 60)

    for lang in languages:
        try:
            # Load metrics from file
            metrics = load_metrics_from_file(lang,flores_dataset,f"metrics/{model_name}")

            if metrics:
                print(f"{lang:15} {float(metrics['BLEU']):8.2f} {float(metrics['chrF']):8.2f} {float(metrics['BERTScore']):10.2f} {float(metrics['COMET']):8.2f}")
            else:
                print(f"{lang:15} {'Error':>8} {'Error':>8} {'Error':>10} {'Error':>8}")

        except Exception as e:
            print(f"Error loading metrics for {lang}: {e}")
            print(f"{lang:15} {'Error':>8} {'Error':>8} {'Error':>10} {'Error':>8}")

    print("-" * 60)

In [ ]:
def zip_folders():
    # Create a timestamp for the zip filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    zip_filename = f"translation_results_{timestamp}.zip"

    # List of folders to zip
    folders_to_zip = [
        "translations",
        "flores101",
        "flores200",
        "floresfixforafrica",
        "metrics"
    ]

    # Create a zip file
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for folder in folders_to_zip:
            # Walk through each folder and add files to the zip
            for root, dirs, files in os.walk(folder):
                for file in files:
                    file_path = os.path.join(root, file)
                    # Add file to zip (maintains directory structure)
                    zipf.write(file_path, os.path.relpath(file_path, start=os.path.dirname(folder)))

    print(f"Successfully created zip archive: {zip_filename}")
    return zip_filename

# Run the function
zip_path = zip_folders()